### GroupChat: Get latest PR from ML.NET

This notebook shows how to use three `GPTAgent` to implement a back-and-force coding-debuging conversation using the `GroupChat` class and get the latest PR from ML.NET.

### Install dependencies

In [1]:
#i "nuget:https://www.myget.org/F/agentchat/api/v3/index.json"

#r "nuget:AgentChat.Core,*-*"
#r "nuget:AgentChat.GPT,*-*"
#r "nuget:AgentChat.Example.Share,*-*"
// this package provides FunctionDefinition for `dotnet-interactive` command
#r "nuget:AgentChat.DotnetInteractiveFunction,*-*"

Restore sources https://www.myget.org/F/agentchat/api/v3/index.json Installed Packages AgentChat.Core, 0.2.4-6632779574 AgentChat.DotnetInteractiveFunction, 0.2.4-6632779574 AgentChat.Example.Share, 0.2.4-6632779574 AgentChat.GPT, 0.2.4-6632779574

### Step 1: Create agents

We are going to create 3 agents
- admin agent: a `GPTAgent` that assign tasks and steps to other two agents.
- coder agent: a `GPTAgent` that will be used to write code on user's task.
- runner agent: a `GPTAgent` that will be used to run the code

In [2]:
using AgentChat.Example.Share;
using AgentChat;
using Azure.AI.OpenAI;

var groupChatFunction = new GroupChatFunction();
var admin = Constant.GPT35.CreateAgent(
    name: "Admin",
    roleInformation: @"You act as group admin that lead other agents to resolve task together. Here's the workflow you follow:
-workflow-
if all_steps_are_resolved
    terminate_chat
else
    resolve_step
-end-

The task is
Retrieve the latest PR from mlnet repo, print the result and save the result to pr.txt.
The steps to resolve the task are:
1. Send a GET request to the GitHub API to retrieve the list of pull requests for the mlnet repo.
2. Parse the response JSON to extract the latest pull request.
3. Print the result to the console and save the result to a file named ""pr.txt"".

Here are some examples for resolve_step:
- The step to resolve is xxx, let's work on this step.",
temperature: 0,
functionMap: new Dictionary<FunctionDefinition, Func<string, Task<string>>>
{
    { groupChatFunction.TerminateGroupChatFunction, groupChatFunction.TerminateGroupChatWrapper }
});

var coder = Constant.GPT35.CreateAgent(
    name: "Coder",
    roleInformation: @"You act as dotnet coder, you write dotnet script to resolve task.
-workflow-
write_code_to_resolve_coding_task

if code_has_error
    fix_code_error

if task_complete, say [COMPLETE]

-end-

Here're some rules to follow on write_code_to_resolve_current_step:
- put code between ```csharp and ```
- Use top-level statements, remove main function, just write code, like what python does.
- Remove all `using` statement. Runner can't handle it.
- Try to use `var` instead of explicit type.
- Try avoid using external library.
- Don't use external data source, like file, database, etc. Create a dummy dataset if you need.
- Always print out the result to console. Don't write code that doesn't print out anything.

Here are some examples for write_code_to_resolve_coding_task:
```nuget
xxx
```
```csharp
xxx
```

Here are some examples for fix_code_error:
The error is caused by xxx. Here's the fix code
```csharp
xxx
```",
    temperature: 0);

Create runner agent

In [3]:
using System.IO;
using AgentChat.DotnetInteractiveService;
using Azure.AI.OpenAI;

var workDir = Path.Combine(Path.GetTempPath(), "InteractiveService");
if (!Directory.Exists(workDir))
    Directory.CreateDirectory(workDir);

var service = new InteractiveService(workDir);
var dotnetInteractiveFunctions = new DotnetInteractiveFunction(service);

// this function is used to fix invalid json returned by GPT-3
var fixInvalidJsonFunction = new FixInvalidJsonFunctionWrapper(Constant.GPT35);

var runner = Constant.GPT35.CreateAgent(
    name: "Runner",
    roleInformation: @"you act as dotnet runner, you run dotnet script and install nuget packages. Here's the workflow you follow:
-workflow-
if code_is_available
    call run_code

if nuget_packages_is_available
    call install_nuget_packages

for any other case
    say [NO_CODE_AVAILABLE]
-end-",
    temperature: 0,
    functionMap: new Dictionary<FunctionDefinition, Func<string, Task<string>>> {
        { dotnetInteractiveFunctions.RunCodeFunction, fixInvalidJsonFunction.FixInvalidJsonWrapper(dotnetInteractiveFunctions.RunCodeWrapper) },
        { dotnetInteractiveFunctions.InstallNugetPackagesFunction, dotnetInteractiveFunctions.InstallNugetPackagesWrapper },
    });

// start kenel
await service.StartAsync(workDir, default);

### Step 2: Create a GroupChat and setup initialize messages

In [4]:
var groupChat = new GroupChat(
    chatLLM: Constant.GPT35,
    admin: admin,
    agents: new List<IAgent> { coder, runner },
    initializeMessages: null);

admin.AddInitializeMessage("Welcome to the group chat! Work together to resolve my task.", groupChat);
coder.AddInitializeMessage("Hey I'm Coder", groupChat);
runner.AddInitializeMessage("Hey I'm Runner", groupChat);
admin.AddInitializeMessage($"The link to mlnet repo is: https://github.com/dotnet/machinelearning. you don't need a token to use github pr api. Make sure to include a User-Agent header, otherwise github will reject it.", groupChat);
admin.AddInitializeMessage(@$"Here's the workflow for this group chat
-groupchat workflow-
if all_steps_are_resolved
    admin_terminate_chat
else

admin_give_step_to_resolve
coder_write_code_to_resolve_step
runner_run_code_from_coder
if code_is_correct
    admin_give_next_step
else
    coder_fix_code_error
", groupChat);

### Step 3: Start group chat

In [5]:
var conversation = await admin.SendMessageToGroupAsync("Here's the first step to resolve: Send a GET request to the GitHub API to retrieve the list of pull requests for the mlnet repo.", groupChat, 30, false);

Message from Coder
--------------------
```csharp
using System;
using System.Net.Http;
using System.Threading.Tasks;

var apiUrl = "https://api.github.com/repos/dotnet/machinelearning/pulls";
var userAgent = "MyApp";

async Task<string> GetPullRequests()
{
    using (var client = new HttpClient())
    {
        client.DefaultRequestHeaders.Add("User-Agent", userAgent);
        var response = await client.GetAsync(apiUrl);
        response.EnsureSuccessStatusCode();
        var responseBody = await response.Content.ReadAsStringAsync();
        return responseBody;
    }
}

var pullRequests = await GetPullRequests();
Console.WriteLine(pullRequests);
```

--------------------

function name: RunCode
raw function call: {
  "code": "using System;\r\nusing System.Net.Http;\r\nusing System.Threading.Tasks;\r\n\r\nvar apiUrl = \"https://api.github.com/repos/dotnet/machinelearning/pulls\";\r\nvar userAgent = \"MyApp\";\r\n\r\nasync Task<string> GetPullRequests()\r\n{\r\n    using (var client = 